# RNN

In [1]:
%%time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST data loading
mnist = input_data.read_data_sets('./MNIST_DATA/', one_hot=True)

# parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 100
hidden_layer_size = 128
training_epochs = 10

# placeholders
X = tf.placeholder(tf.float32, 
                   shape=[None, time_steps, element_size], name='inputs')
Y_ = tf.placeholder(tf.float32, 
                   shape=[None, num_classes], name='inputs')

# RNN cell
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)

# Output Layer Weights
Wl = tf.Variable(tf.truncated_normal(
                    [hidden_layer_size, num_classes],
                    mean=0, stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes], 
                                     mean=0, stddev=.01))

def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl

# Output
last_rnn_output = outputs[:, -1, :]
final_output = get_linear_layer(last_rnn_output)

# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=final_output,labels=Y_))

# optimization (RMSProp)
train_step = tf.train.RMSPropOptimizer(
                        0.001, 0.9).minimize(cross_entropy)

# accuracy
correct_prediction = tf.equal(tf.argmax(Y_, 1), 
                              tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, 
                                   tf.float32)))*100


# Test data
#test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
#test_labels = mnist.test.labels[:batch_size]

# session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# train
for epoch in range(training_epochs):
    batch_count = int(mnist.train.num_examples/batch_size)
    for i in range(batch_count):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape(
            (batch_size, time_steps, element_size))
        _, acc = sess.run([train_step, accuracy], 
                             feed_dict={X: batch_x,
                                        Y_: batch_y})
    print("Epoch: ", epoch, "Train Accuracy: ", acc)
   
# Test
test_images = mnist.test.images.reshape((-1, time_steps, element_size))

print("Test Accuracy: ", sess.run(accuracy,
                                 feed_dict={X: test_images,
                                            Y_: mnist.test.labels}))
print("done")
sess.close()

Extracting ./MNIST_DATA/train-images-idx3-ubyte.gz
Extracting ./MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting ./MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch:  0 Train Accuracy:  87.0
Epoch:  1 Train Accuracy:  95.0
Epoch:  2 Train Accuracy:  94.0
Epoch:  3 Train Accuracy:  96.0
Epoch:  4 Train Accuracy:  95.0
Epoch:  5 Train Accuracy:  98.0
Epoch:  6 Train Accuracy:  97.0
Epoch:  7 Train Accuracy:  97.0
Epoch:  8 Train Accuracy:  94.0
Epoch:  9 Train Accuracy:  99.0
Test Accuracy:  97.08
done
Wall time: 3min 27s
